In [171]:
import pandas as pd
df = pd.read_excel(r'NewNewNew(1).xlsx')
df

,公开(公告)号,[标]原始申请(专利权)人,原始申请(专利权)人地址,申请日,IPC主分类号,类别,Year
0,US62875872P0,拜爾拉吉克股份有限公司,US,2019,-,未分类,2
1,US62872589P0,拜爾拉吉克股份有限公司,US,2019,-,未分类,2
2,US675631A,FARBENFABEN OF ELBERFELD,-,1900,C07C,聚合物及其制造工艺,1
3,US20230250275A1,ALGIX,US,2023,C08L,聚合物及其制造工艺,3
4,US20230212537A1,CARBIOS,FR,2022,C12N,微生物或酶、其组合物,3
...,...,...,...,...,...,...,...
445,EP2294125A2,罗盖特公司,FR,2009,C08L,聚合物及其制造工艺,1
446,EP2247661A2,罗盖特公司,FR,2009,C08L,聚合物及其制造工艺,1
447,EP2144221B1,株式会社LG化学,KR,2009,G09F,未分类,1
448,EP2144966A1,法国阿科玛公司,FR,2008,C08L,聚合物及其制造工艺,1


In [172]:
import pandas as pd
import numpy as np

# 将申请日列转换为datetime格式，并提取年份作为新的一列
df['year'] = df['Year']

# 按照年份和IPC主分类号分组，统计每个年份和IPC主分类号下的专利数量
grouped = df.groupby(['year', 'IPC主分类号'])['IPC主分类号'].count().reset_index(name='count')

# 根据专利数量将数据框分组，并计算每组中的年份数量
bins = [-np.inf, 0, 1, 10, 20, np.inf]
levels = ['L1', 'L2', 'L3', 'L4', 'L5']
grouped['label'] = pd.cut(grouped['count'], bins=bins, labels=levels)
result = grouped.groupby(['IPC主分类号', 'year'])['label'].first().unstack(fill_value='L1')
result = result.transpose()
result

IPC主分类号,-,A01G,A01N,A24D,A61K,A61L,A62B,A63H,B01D,B01J,...,D01D,D01F,D02G,D03D,D04H,D21H,F42B,G06F,G09F,H02J
year,,,,,,,,,,,,,,,,,,,,,
1,L1,L1,L1,L3,L3,L2,L1,L1,L1,L1,...,L1,L1,L1,L1,L2,L2,L1,L2,L2,L1
2,L3,L3,L1,L3,L3,L1,L1,L2,L1,L2,...,L1,L3,L2,L2,L1,L3,L2,L3,L3,L2
3,L1,L2,L3,L2,L3,L1,L2,L1,L3,L1,...,L3,L3,L1,L2,L2,L2,L1,L1,L1,L2


In [173]:
result.to_excel('resultIPC分阶段_5类.xlsx')

In [174]:
import pandas as pd
df = pd.read_excel(r'resultIPC分阶段_5类.xlsx')
df

,year,-,A01G,A01N,A24D,A61K,A61L,A62B,A63H,B01D,...,D01D,D01F,D02G,D03D,D04H,D21H,F42B,G06F,G09F,H02J
0,1,L1,L1,L1,L3,L3,L2,L1,L1,L1,...,L1,L1,L1,L1,L2,L2,L1,L2,L2,L1
1,2,L3,L3,L1,L3,L3,L1,L1,L2,L1,...,L1,L3,L2,L2,L1,L3,L2,L3,L3,L2
2,3,L1,L2,L3,L2,L3,L1,L2,L1,L3,...,L3,L3,L1,L2,L2,L2,L1,L1,L1,L2


In [175]:
# 将year列转换为datetime格式，并提取年份作为新的一列
LF = ['L1', 'L2', 'L3', 'L4', 'L5']
LR = ['L1', 'L2', 'L3', 'L4', 'L5']

In [176]:
# 创建转移矩阵
transition_matrix = np.zeros((len(levels), len(levels)))

In [177]:
import numpy as np
# 计算转移概率
def compute_transition_matrix(t, T, LF, LR):
    transition_matrix = np.zeros((len(LF), len(LR)))
    for i in range(len(LF)):
        for j in range(len(LR)):
            ma = 0
            columns = []
            for column in df.columns:
                if column != 'year':
                    if (df[df['year'] == t][column] == LF[i]).any():
                        ma += 1
                        columns.append(column)
            mab = 0
            for column in columns:
                mab += (df[df['year'] == T][column] == LR[j]).sum()
            if ma == 0:
                Mab = 0
            else:
                Mab = mab / ma
            transition_matrix[i, j] = Mab
    return transition_matrix

In [178]:
# 计算转移矩阵
t = 1
T = 2
transition_matrix = compute_transition_matrix(t, T, LF, LR)

In [179]:
Markov_matrix = pd.DataFrame(transition_matrix, index=levels, columns=levels)
Markov_matrix

,L1,L2,L3,L4,L5
L1,0.250000,0.500000,0.250000,0.000000,0.000000
L2,0.272727,0.000000,0.545455,0.181818,0.000000
L3,0.000000,0.166667,0.500000,0.000000,0.333333
L4,0.000000,0.000000,0.000000,0.000000,1.000000
L5,0.000000,0.000000,0.000000,0.000000,0.000000


In [180]:
# 计算转移矩阵
t2 = 2
T2 = 3
transition_matrix2 = compute_transition_matrix(t2, T2, LF, LR)

In [181]:
Markov_matrix2 = pd.DataFrame(transition_matrix2, index=levels, columns=levels)
Markov_matrix2

,L1,L2,L3,L4,L5
L1,0.200000,0.4000,0.400000,0.000000,0.000000
L2,0.533333,0.4000,0.066667,0.000000,0.000000
L3,0.375000,0.3125,0.250000,0.062500,0.000000
L4,0.000000,0.0000,1.000000,0.000000,0.000000
L5,0.000000,0.0000,0.000000,0.333333,0.666667


In [182]:
Markov_matrix.to_csv('Markov_matrixIPC分阶段_5类.csv')

In [183]:
levels = ['L1', 'L2', 'L3', 'L4', 'L5']

columns = ['year', 'L1', 'L2', 'L3', 'L4', 'L5']
df2 = pd.DataFrame(columns=columns)

for i in range(0,len(df)):
    row = df.iloc[i]
    year = row.iloc[0]
    L1, L2, L3, L4, L5 = 0, 0, 0, 0, 0
    for i in range(1,len(row)):
        level = row.iloc[i]
        if level == levels[0]:
            L1 = L1 + 1
        if level == levels[1]:
            L2 = L2 + 1
        if level == levels[2]:
            L3 = L3 + 1
        if level == levels[3]:
            L4 = L4 + 1
        if level == levels[4]:
            L5 = L5 + 1
    new_row = {'year': year, 'L1': L1, 'L2': L2, 'L3': L3, 'L4': L4, 'L5': L5}   
    df2 = pd.concat([df2, pd.DataFrame(new_row, index=[0])], ignore_index=True)

In [184]:
df2

,year,L1,L2,L3,L4,L5
0,1,28,11,6,1,0
1,2,10,15,16,2,3
2,3,16,15,11,2,2


In [185]:
df2.to_excel('IPC分布分阶段_5类.xlsx')

In [190]:
def identify_transitions(df, year1, year2):

    df_year1 = df[df['year'] == year1].iloc[0]
    df_year2 = df[df['year'] == year2].iloc[0]

    positive_transitions = []
    negative_transitions = []

    levels = {'L1': 1, 'L2': 2, 'L3': 3, 'L4': 4, 'L5': 5}

    for column in df.columns[1:]:
        level_year1 = levels[df_year1[column]]
        level_year2 = levels[df_year2[column]]

        if level_year1 < level_year2:
            positive_transitions.append(f"{column}: L{level_year1} -> L{level_year2}")
        elif level_year1 > level_year2:
            negative_transitions.append(f"{column}: L{level_year1} -> L{level_year2}")

    return positive_transitions, negative_transitions

In [191]:
df

,year,-,A01G,A01N,A24D,A61K,A61L,A62B,A63H,B01D,...,D01D,D01F,D02G,D03D,D04H,D21H,F42B,G06F,G09F,H02J
0,1,L1,L1,L1,L3,L3,L2,L1,L1,L1,...,L1,L1,L1,L1,L2,L2,L1,L2,L2,L1
1,2,L3,L3,L1,L3,L3,L1,L1,L2,L1,...,L1,L3,L2,L2,L1,L3,L2,L3,L3,L2
2,3,L1,L2,L3,L2,L3,L1,L2,L1,L3,...,L3,L3,L1,L2,L2,L2,L1,L1,L1,L2


In [192]:
data = pd.read_excel('resultIPC分阶段_5类.xlsx')

positive, negative = identify_transitions(data, 1, 2)

print("正向转移:", positive)
print("负向转移:", negative)

正向转移: ['-: L1 -> L3', 'A01G: L1 -> L3', 'A63H: L1 -> L2', 'B01J: L1 -> L2', 'B05D: L1 -> L2', 'B09B: L1 -> L2', 'B26B: L1 -> L3', 'B27N: L1 -> L2', 'B29B: L1 -> L2', 'B29C: L2 -> L3', 'B32B: L3 -> L5', 'B65D: L2 -> L4', 'B65F: L1 -> L2', 'C07C: L2 -> L3', 'C07D: L1 -> L3', 'C08B: L1 -> L2', 'C08F: L1 -> L2', 'C08G: L2 -> L4', 'C08J: L3 -> L5', 'C08L: L4 -> L5', 'C09D: L2 -> L3', 'C12N: L1 -> L3', 'C12P: L1 -> L3', 'C23C: L1 -> L2', 'D01F: L1 -> L3', 'D02G: L1 -> L2', 'D03D: L1 -> L2', 'D21H: L2 -> L3', 'F42B: L1 -> L2', 'G06F: L2 -> L3', 'G09F: L2 -> L3', 'H02J: L1 -> L2']
负向转移: ['A61L: L2 -> L1', 'C08H: L2 -> L1', 'C09J: L3 -> L2', 'D04H: L2 -> L1']


In [193]:
data = pd.read_excel('resultIPC分阶段_5类.xlsx')

positive, negative = identify_transitions(data, 2, 3)

print("正向转移:", positive)
print("负向转移:", negative)

正向转移: ['A01N: L1 -> L3', 'A62B: L1 -> L2', 'B01D: L1 -> L3', 'B09B: L2 -> L3', 'C02F: L1 -> L2', 'C05F: L1 -> L2', 'C08K: L3 -> L4', 'C09C: L1 -> L3', 'D01D: L1 -> L3', 'D04H: L1 -> L2']
负向转移: ['-: L3 -> L1', 'A01G: L3 -> L2', 'A24D: L3 -> L2', 'A63H: L2 -> L1', 'B01J: L2 -> L1', 'B05D: L2 -> L1', 'B26B: L3 -> L2', 'B27N: L2 -> L1', 'B32B: L5 -> L4', 'B65D: L4 -> L3', 'B65F: L2 -> L1', 'C07C: L3 -> L2', 'C07D: L3 -> L1', 'C08G: L4 -> L3', 'C09D: L3 -> L1', 'C12P: L3 -> L1', 'C23C: L2 -> L1', 'D02G: L2 -> L1', 'D21H: L3 -> L2', 'F42B: L2 -> L1', 'G06F: L3 -> L1', 'G09F: L3 -> L1']
